In [21]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Завантажити дані, вивести назви колонок і розмір датасета

In [24]:
df = pd.read_csv("bodyfat.csv", encoding='windows-1252')

# Додав ще одну колонку для присвоєння спрощеного індекшу ожиріння
bin_labels = [0, 1, 2, 3]

df['BodyFatEncoded'] = pd.cut(df['BodyFat'], bins=4, labels=bin_labels)

print(f'Dataset size: {df.shape}')
df.head()


Dataset size: (252, 16)


,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist,BodyFatEncoded
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1,1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2,0
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6,2
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2,0
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7,2


## Опрацювати пропуски

In [25]:
df.info()
df_cleaned = df.dropna()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Density         252 non-null    float64 
 1   BodyFat         252 non-null    float64 
 2   Age             252 non-null    int64   
 3   Weight          252 non-null    float64 
 4   Height          252 non-null    float64 
 5   Neck            252 non-null    float64 
 6   Chest           252 non-null    float64 
 7   Abdomen         252 non-null    float64 
 8   Hip             252 non-null    float64 
 9   Thigh           252 non-null    float64 
 10  Knee            252 non-null    float64 
 11  Ankle           252 non-null    float64 
 12  Biceps          252 non-null    float64 
 13  Forearm         252 non-null    float64 
 14  Wrist           252 non-null    float64 
 15  BodyFatEncoded  252 non-null    category
dtypes: category(1), float64(14), int64(1)
memory usage: 30.1 KB
<c

## Нормалізувати дані

In [26]:
scaler = preprocessing.MinMaxScaler()
df_cleaned_without_code = df_cleaned.drop("BodyFatEncoded", axis=1)
columns_names = df_cleaned_without_code.columns
dt_transform = scaler.fit_transform(df_cleaned_without_code)
normalized_dataset = pd.DataFrame(dt_transform, columns=columns_names)
#Розділяю дані на характеристики та результат який ми будемо вгадувати
characteristics = normalized_dataset
result = df_cleaned["BodyFatEncoded"]


### 1. Повнозв'язані нейронні мережі
Вирішіть завдання класифікації даних, з якими ви працювали в лабораторній № 1 за допомогою повнозв’язаної нейромережі прямого поширення (fully connected feed-forward network). Результати порівняйте з одержаними раніше.

In [27]:
characteristics_train, characteristics_test, result_train, result_test = train_test_split(characteristics,result, test_size=0.3,stratify=result)

lb = LabelBinarizer()
trainResult = lb.fit_transform(result_train)
testResult = lb.transform(result_test)

characteristics_train.shape

testResult[:2]

array([[1, 0, 0, 0],
       [1, 0, 0, 0]])

In [28]:
# Навчання мережі
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
model = Sequential()
model.add(Dense(256, input_shape=(15,), activation="sigmoid"))#1th hiddenlayer
model.add(Dense(128, activation="sigmoid"))#2th hiddenlayer
model.add(Dense(4, activation="softmax"))#outputlayer

sgd = SGD(0.1)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
    metrics=["accuracy"])
H = model.fit(characteristics_train, trainResult, validation_data=(characteristics_test, testResult),
    epochs=200, batch_size=8)


Epoch 1/200
22/22 [==============================] - 1s 22ms/step - loss: 1.3570 - accuracy: 0.3693 - val_loss: 1.1473 - val_accuracy: 0.4737
Epoch 2/200
22/22 [==============================] - 0s 3ms/step - loss: 1.3288 - accuracy: 0.3864 - val_loss: 1.1297 - val_accuracy: 0.4737
Epoch 3/200
22/22 [==============================] - 0s 3ms/step - loss: 1.1490 - accuracy: 0.4886 - val_loss: 1.1795 - val_accuracy: 0.4737
Epoch 4/200
22/22 [==============================] - 0s 3ms/step - loss: 1.1982 - accuracy: 0.4091 - val_loss: 1.1247 - val_accuracy: 0.4737
Epoch 5/200
22/22 [==============================] - 0s 3ms/step - loss: 1.2092 - accuracy: 0.4148 - val_loss: 1.1221 - val_accuracy: 0.6184
Epoch 6/200
22/22 [==============================] - 0s 3ms/step - loss: 1.2441 - accuracy: 0.3580 - val_loss: 1.1184 - val_accuracy: 0.4737
Epoch 7/200
22/22 [==============================] - 0s 3ms/step - loss: 1.1397 - accuracy: 0.4205 - val_loss: 1.1549 - val_accuracy: 0.6447
Epoch 8/200


In [33]:
# Оцінка мережі
predictions = model.predict(characteristics_test, batch_size=8)
print(classification_report(testResult.argmax(axis=1),
    predictions.argmax(axis=1),
    target_names=[str(x) for x in lb.classes_]))

# argmax(axis=1) використовується для отримання індексу класу, який має найвищу ймовірність для кожного прикладу в тестовому наборі.
# predictions.argmax(axis=1): Аналогічно, це отримує індекс класу з передбачень моделі, який має найвищу ймовірність.

10/10 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.94      0.88      0.91        17
           1       0.94      0.92      0.93        36
           2       0.88      1.00      0.94        22
           3       0.00      0.00      0.00         1

    accuracy                           0.92        76
   macro avg       0.69      0.70      0.69        76
weighted avg       0.91      0.92      0.91        76



C:\Users\m.shovak\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\m.shovak\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\m.shovak\AppData\Local\Programs\Python\Python310\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

### Згорткові нейронні мережі
Вирішіть завдання класифікації зображень за допомогою згорткової (convolutional) нейромережі (якщо в обраному датасеті класів забагато, достатньо залишити 3-5).

In [17]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


balls_train = tf.keras.utils.image_dataset_from_directory(
    "C:\\Users\\m.shovak\\Sport-balls", validation_split=0.3, subset="training", seed=42, image_size=(150, 150),
    batch_size=32, label_mode="int")

balls_test = tf.keras.utils.image_dataset_from_directory(
    "C:\\Users\\m.shovak\\Sport-balls", validation_split=0.3, subset="validation", seed=42, image_size=(150, 150),
    batch_size=32, label_mode="int")

Found 689 files belonging to 6 classes.
Using 483 files for training.
Found 689 files belonging to 6 classes.
Using 206 files for validation.


In [20]:
balls_train.class_names
# Preprocess the data (normalize pixel values)
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
balls_normilized = balls_train.map(lambda x, y: (normalization_layer(x), y))

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(15, activation='softmax')
])

opt = Adam(learning_rate=0.001)
# Compile the model
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
#early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

model.fit(balls_train, epochs=50, batch_size=15, validation_data=balls_test)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 conv2d_27 (Conv2D)          (None, 146, 146, 64)      18496     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 73, 73, 64)        0         
 ng2D)                                                           
                                                                 
 dropout_26 (Dropout)        (None, 73, 73, 64)        0         
                                                                 
 flatten_13 (Flatten)        (None, 341056)            0         
                                                                 
 dense_26 (Dense)            (None, 128)               43655296  
                                                     

### Рекурентні нейронні мережі
Вирішіть задачу класифікації текстів (з якими ви працювали в лабораторній № 2) за допомогою рекурентної нейромережі. Результати порівняйте з одержаними раніш.

In [29]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer


news_dataset = pd.read_csv("news_articles.csv", encoding='windows-1252')
# Delete unnecessary columns
news_eraced = news_dataset[['text', 'label']]
# Delete empty falues
news_eraced= news_eraced.dropna()
stop_words = stopwords.words('english')


def text_preprocessing(text):
    text = text.lower()
    wordnet_lemmatizer = WordNetLemmatizer()
    word_tokens = word_tokenize(text)
    for i in word_tokens:
        if i not in stop_words:
                # remove punctuations
                text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
                # remove extra spaces
                text = re.sub('\n', ' ', text)
                text = re.sub('\s{2,}',' ', text)
                text = wordnet_lemmatizer.lemmatize(text)
                # remove non-Latin characters
                text = re.sub('[^a-zA-Z\s]', '', text)
    return text
news_eraced.text = news_eraced.text.apply(text_preprocessing)

text_train, text_test, label_train, label_test = train_test_split(news_eraced.text, news_eraced.label, test_size=0.2)


# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(text_train)
sequences = tokenizer.texts_to_sequences(text_train)

# Pad the sequences so they all have the same length
text_train = pad_sequences(sequences, maxlen=150)
text_test = pad_sequences(tokenizer.texts_to_sequences(text_test), maxlen=150)

# Create one-hot encoded labels
label_train = pd.get_dummies(label_train).values
y_test = pd.get_dummies(label_test).values


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\m.shovak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\m.shovak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\m.shovak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
from tensorflow.python.layers import layers
from tensorflow.keras.layers import Embedding, Flatten,Dropout,SimpleRNN,LSTM,GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.optimizers import Adam

# Build the model
model = Sequential()
model.add(Embedding(1000, 64, input_length=150))
model.add(SpatialDropout1D(0.3))
#model.add(SimpleRNN(64,return_sequences=False))#acc=0.6159
#model.add(GRU(64,return_sequences=False))#acc=0.6159
model.add(LSTM(64,return_sequences=False))
model.add(layers.Dropout(0.2))
model.add(Dense(2, activation='softmax'))


opt = Adam(learning_rate=0.0001)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Train the model
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

history = model.fit(text_train, label_train, batch_size=4, epochs=20,validation_split=0.2, verbose=1,callbacks=[early_stop])


Epoch 1/20
328/328 [==============================] - 11s 28ms/step - loss: 0.6730 - accuracy: 0.6136 - val_loss: 0.6544 - val_accuracy: 0.6311
Epoch 2/20
328/328 [==============================] - 9s 27ms/step - loss: 0.6514 - accuracy: 0.6265 - val_loss: 0.6399 - val_accuracy: 0.6311
Epoch 3/20
328/328 [==============================] - 9s 27ms/step - loss: 0.6268 - accuracy: 0.6311 - val_loss: 0.6124 - val_accuracy: 0.6372
Epoch 4/20
328/328 [==============================] - 9s 27ms/step - loss: 0.6021 - accuracy: 0.6692 - val_loss: 0.6059 - val_accuracy: 0.6707
Epoch 5/20
328/328 [==============================] - 9s 27ms/step - loss: 0.5464 - accuracy: 0.7073 - val_loss: 0.5810 - val_accuracy: 0.6646
Epoch 6/20
328/328 [==============================] - 9s 27ms/step - loss: 0.4886 - accuracy: 0.7774 - val_loss: 0.5889 - val_accuracy: 0.6646
Epoch 7/20
328/328 [==============================] - 9s 28ms/step - loss: 0.4351 - accuracy: 0.7957 - val_loss: 0.6311 - val_accuracy: 0.686